<a href="https://colab.research.google.com/github/AlekhyaGangopadhyay/IITPatna_Internship_LLMs/blob/main/InternVideo2_5_4B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
#  InternVideo2.5-8B  ▸  Finance Harassment/Abuse Dataset
#  Full Pipeline : Train (LoRA) → Infer → Accuracy Score
#  Dataset       : iamalekhya/Finance_set  (318 videos, 9 cols)
#
#  InternVideo2.5 is built ON TOP of InternVL2.5.
#  It adds HiCo token compression + TPO for long video understanding.
#  Uses the same model.chat() API as InternVL2.5 but adds
#  num_patches_list to handle dynamic tiling per frame.
#
#  GPU:  ~16 GB in bfloat16  →  T4 free Colab ✅  (with 4-bit)
#        ~14 GB in 4-bit     →  T4 free Colab ✅  ← recommended
# ============================================================

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 1 ── Install  (run once → Runtime ▸ Restart session)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# IMPORTANT: transformers must be pinned to 4.40.1 for InternVideo2.5
!pip install "transformers==4.40.1" accelerate sentencepiece peft bitsandbytes
!pip install decord av imageio opencv-python
!pip install rouge-score openpyxl scikit-learn
!pip install flash-attn --no-build-isolation   # optional but recommended

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 2 ── Mount Google Drive
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
from google.colab import drive
drive.mount('/content/drive')

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 3 ── ⚙️  CONFIG  ← only edit this section
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

MODEL_PATH = "OpenGVLab/InternVideo2_5_Chat_4B"

# Dataset
HF_DATASET     = "iamalekhya/Finance_set"
EXCEL_FALLBACK = "/content/drive/MyDrive/IITP_Internship/Finance_set.xlsx"

# Videos in Google Drive
VIDEO_DIRS = [
    "/content/drive/Shared drives/Finance_video_Anamoly/AG",
    "/content/drive/Shared drives/Finance_video_Anamoly/ag",
    "/content/drive/Shared drives/Finance_video_Anamoly/Alekhya",
    "/content/drive/Shared drives/Finance_video_Anamoly/Pritam",
    "/content/drive/Shared drives/Finance_video_Anamoly/Videos",
]
VIDEO_EXT = ".mp4"

# Training
# InternVideo2.5 uses dynamic tiling: each frame → multiple tiles (num_patches_list)
# max_num=1 means 1 tile + 1 thumbnail = 2 patches per frame → safer for T4
NUM_SEGMENTS = 16    # number of frames to sample from each video
MAX_NUM      = 1     # max dynamic tiles per frame  (1 = safe for T4, up to 6 for A100)
IMAGE_SIZE   = 448   # InternVideo2.5 native resolution
MAX_SEQ_LEN  = 2048
EPOCHS       = 3
BATCH_SIZE   = 1
GRAD_ACCUM   = 8
LORA_RANK    = 16
EVAL_SPLIT   = 0.10
RANDOM_SEED  = 42

# Paths
OUTPUT_DIR          = "./internvideo25_finance"
ADAPTER_DRIVE       = "/content/drive/MyDrive/IIT_Internship/internvideo25_8b_adapter"
RESULTS_EXCEL_DRIVE = "/content/drive/MyDrive/IIT_Internship/internvideo25_8b_results.xlsx"

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 4 ── Imports
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import os, re, json, shutil, torch, warnings
import numpy as np
import pandas as pd
import torchvision.transforms as T
from PIL import Image
from decord import VideoReader, cpu
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from rouge_score import rouge_scorer as rs
from torch.utils.data import Dataset as TorchDataset
from torch.nn.utils.rnn import pad_sequence
from transformers import TrainingArguments, Trainer

import decord
decord.bridge.set_bridge("torch")

warnings.filterwarnings("ignore")
print("✅ Imports OK")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 5 ── Official InternVideo2.5 Image Preprocessing
#            (dynamic tiling — official code from HuggingFace)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

def build_transform(input_size=IMAGE_SIZE):
    return T.Compose([
        T.Lambda(lambda img: img.convert("RGB") if img.mode != "RGB" else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
    ])

TRANSFORM = build_transform(IMAGE_SIZE)


def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float("inf")
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio


def dynamic_preprocess(image, min_num=1, max_num=MAX_NUM,
                        image_size=IMAGE_SIZE, use_thumbnail=True):
    """
    Split a PIL image into dynamic tiles based on aspect ratio.
    Always adds a thumbnail tile when use_thumbnail=True.
    Returns list of PIL tile images.
    """
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    target_ratios = set(
        (i, j)
        for n in range(min_num, max_num + 1)
        for i in range(1, n + 1)
        for j in range(1, n + 1)
        if min_num <= i * j <= max_num
    )
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size
    )

    target_width  = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks        = target_aspect_ratio[0] * target_aspect_ratio[1]

    resized_img       = image.resize((target_width, target_height))
    processed_images  = []

    for i in range(blocks):
        cols = target_width // image_size
        box  = (
            (i % cols) * image_size,
            (i // cols) * image_size,
            ((i % cols) + 1) * image_size,
            ((i // cols) + 1) * image_size,
        )
        processed_images.append(resized_img.crop(box))

    if use_thumbnail and len(processed_images) != 1:
        processed_images.append(image.resize((image_size, image_size)))

    return processed_images


def process_pil_frame(pil_img, max_num=MAX_NUM) -> tuple:
    """
    Convert a single PIL frame → (pixel_values_tensor, num_patches).
    num_patches = number of tiles generated for this frame.
    """
    tiles        = dynamic_preprocess(pil_img, max_num=max_num, use_thumbnail=True)
    pixel_values = torch.stack([TRANSFORM(tile) for tile in tiles])
    return pixel_values, pixel_values.shape[0]   # (n_tiles, 3, H, W), n_tiles


def load_video_internvideo(video_path: str,
                            num_segments: int = NUM_SEGMENTS,
                            max_num: int = MAX_NUM) -> tuple:
    """
    Official InternVideo2.5 video loading using decord.
    Returns (pixel_values, num_patches_list):
      pixel_values     : (total_tiles, 3, H, W)
      num_patches_list : list of tile counts per frame — required by model.chat()
    """
    vr        = VideoReader(video_path, ctx=cpu(0), num_threads=1)
    max_frame = len(vr) - 1
    fps       = float(vr.get_avg_fps())

    seg_size     = float(max_frame) / num_segments
    frame_indices = np.array([
        int(seg_size / 2 + np.round(seg_size * i))
        for i in range(num_segments)
    ])
    frame_indices = np.clip(frame_indices, 0, max_frame)

    pixel_values_list = []
    num_patches_list  = []

    for idx in frame_indices:
        pil_img     = Image.fromarray(vr[idx].asnumpy()).convert("RGB")
        pv, n_tiles = process_pil_frame(pil_img, max_num=max_num)
        pixel_values_list.append(pv)
        num_patches_list.append(n_tiles)

    pixel_values = torch.cat(pixel_values_list, dim=0)   # (total_tiles, 3, H, W)
    return pixel_values, num_patches_list

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 6 ── Load Dataset
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("Loading dataset...")
try:
    ds = load_dataset(HF_DATASET, split="train")
    df = ds.to_pandas()
    print(f"✅ Loaded from HuggingFace: {len(df)} rows")
except Exception as e:
    print(f"⚠️  HF failed ({e}), loading from Excel...")
    df = pd.read_excel(EXCEL_FALLBACK)
    print(f"✅ Loaded from Excel: {len(df)} rows")

df.columns = [c.strip() for c in df.columns]
REQUIRED = ["uni_id","Category","Aspect","Intent","Explanation","Country","Source","Sector"]
missing  = [c for c in REQUIRED if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}")

df = df.dropna(subset=REQUIRED).reset_index(drop=True)
print(f"   Rows: {len(df)} | Categories: {df['Category'].unique().tolist()}")

ALL_ASPECTS = sorted(df["Aspect"].unique().tolist())
print(f"   Aspects: {len(ALL_ASPECTS)} unique values")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 7 ── Prompt
#
#  InternVideo2.5 video format (official):
#    video_prefix = "Frame1: <image>\nFrame2: <image>\n..."
#    question     = video_prefix + instruction_text
#    num_patches_list is passed to model.chat() separately
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
def build_question(num_frames: int) -> str:
    """
    Build question string with Frame{i}: <image> prefix for each frame.
    This is the EXACT format required by InternVideo2.5.
    """
    video_prefix = "".join([f"Frame{i+1}: <image>\n" for i in range(num_frames)])

    instruction = f"""You are an expert video analyst specialising in financial harassment, fraud, abuse, and workplace misconduct.

The frames above are sampled evenly from a video. Analyse them carefully and return a single valid JSON object with EXACTLY these fields:
{{
  "category"   : "Harassment" | "Bullying",
  "aspect"     : one of the known aspect labels listed below,
  "intent"     : 1 (intentional) | 0 (unintentional),
  "explanation": "1-2 sentence description of what is happening and why it is problematic",
  "country"    : "country name or Unknown",
  "source"     : "YouTube | News | Training | Documentary | Other",
  "sector"     : "Finance | Healthcare | Education | Retail | Tech | Government | General"
}}

Known aspect labels (pick the closest match):
{json.dumps(ALL_ASPECTS, indent=2)}

Rules:
- Output ONLY the JSON. No preamble, no markdown fences, no extra text.
- Use double quotes for all strings.
- intent must be an integer (1 or 0), not a string.

Observe carefully:
• Who is involved and what is happening
• The type and form of misconduct
• Whether the act appears deliberate
• Country and industry context clues"""

    return video_prefix + instruction

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 8 ── Helpers
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
def find_video(uni_id: str) -> str:
    filename = f"{uni_id}{VIDEO_EXT}"
    for folder in VIDEO_DIRS:
        path = os.path.join(folder, filename)
        if os.path.exists(path):
            return path
        if os.path.isdir(folder):
            for f in os.listdir(folder):
                if f.lower() == filename.lower():
                    return os.path.join(folder, f)
    return ""


def row_to_json(row) -> str:
    return json.dumps({
        "category"   : str(row["Category"]).strip(),
        "aspect"     : str(row["Aspect"]).strip(),
        "intent"     : int(row["Intent"]),
        "explanation": str(row["Explanation"]).strip(),
        "country"    : str(row["Country"]).strip(),
        "source"     : str(row["Source"]).strip(),
        "sector"     : str(row["Sector"]).strip(),
    }, indent=2)


def parse_json_response(text: str) -> dict:
    try:
        m = re.search(r'\{.*\}', text, re.DOTALL)
        if m:
            return json.loads(m.group())
    except Exception:
        pass
    return {}

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 9 ── Load Model + Tokenizer + LoRA
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print(f"\nLoading {MODEL_PATH}...")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code = True,
    use_fast          = False,
)

# 4-bit quantisation — fits on free T4 (16 GB)
# For A100: remove quant_config and use torch_dtype=torch.bfloat16
quant_config = BitsAndBytesConfig(
    load_in_4bit              = True,
    bnb_4bit_quant_type       = "nf4",
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype    = torch.bfloat16,
)

model = AutoModel.from_pretrained(
    MODEL_PATH,
    quantization_config = quant_config,
    trust_remote_code   = True,
    low_cpu_mem_usage   = True,
).eval()

print("✅ Model loaded")

# LoRA on the InternLM2 language backbone
# InternVideo2.5 language model is an InternLM2-7B accessed via .language_model
lora_config = LoraConfig(
    task_type      = TaskType.CAUSAL_LM,
    r              = LORA_RANK,
    lora_alpha     = LORA_RANK * 2,
    lora_dropout   = 0.05,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    bias           = "none",
)
model.language_model = get_peft_model(model.language_model, lora_config)
model.language_model.print_trainable_parameters()
print("✅ LoRA applied")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 10 ── Build Samples
#             Load video via decord → get pixel_values + num_patches_list
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
train_df, eval_df = train_test_split(df, test_size=EVAL_SPLIT, random_state=RANDOM_SEED)
train_df = train_df.reset_index(drop=True)
eval_df  = eval_df.reset_index(drop=True)
print(f"\nSplit → Train: {len(train_df)}  |  Eval: {len(eval_df)}")


def build_samples(dataframe, split_name="train"):
    samples, skipped = [], 0
    for i, row in dataframe.iterrows():
        uni_id     = str(row["uni_id"]).strip()
        video_path = find_video(uni_id)

        if not video_path:
            print(f"  ⚠️  [{split_name}] Skipping {uni_id} — video not found")
            skipped += 1
            continue

        try:
            pixel_values, num_patches_list = load_video_internvideo(
                video_path, num_segments=NUM_SEGMENTS, max_num=MAX_NUM
            )
        except Exception as e:
            print(f"  ⚠️  [{split_name}] Skipping {uni_id} — decord error: {e}")
            skipped += 1
            continue

        target = row_to_json(row)

        samples.append({
            "pixel_values"    : pixel_values,       # (total_tiles, 3, H, W) float32 tensor
            "num_patches_list": num_patches_list,    # [n_tiles_per_frame, ...]
            "question"        : build_question(len(num_patches_list)),
            "target_json"     : target,
            "uni_id"          : uni_id,
        })

        if (i + 1) % 30 == 0:
            print(f"  [{split_name}] {i+1}/{len(dataframe)}  skipped={skipped}")

    print(f"  ✅ [{split_name}] ready={len(samples)}  skipped={skipped}")
    return samples


print("\n📦 Building training samples (loading videos with decord)...")
train_samples = build_samples(train_df, "train")

print("\n📦 Building eval samples...")
eval_samples  = build_samples(eval_df,  "eval")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 11 ── PyTorch Dataset + Collator
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
IMG_CONTEXT_TOKEN = '<IMG_CONTEXT>'
IMG_TOKEN         = '<image>'

class InternVideo25Dataset(TorchDataset):
    """
    Tokenises using InternVL/InternVideo2.5 chat template.

    Each <image> in the question must be replaced by
    num_image_token copies of <IMG_CONTEXT> for that specific frame.
    Because InternVideo2.5 uses dynamic tiling, different frames may
    have different num_patches, so we expand frame-by-frame.
    """
    def __init__(self, samples, tokenizer, model, max_length=2048):
        self.samples         = samples
        self.tokenizer       = tokenizer
        self.max_length      = max_length
        self.num_image_token = model.num_image_token  # 256 for ViT-448

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s                = self.samples[idx]
        pixel_values     = s["pixel_values"]       # (total_tiles, 3, H, W)
        num_patches_list = s["num_patches_list"]   # [tiles_per_frame, ...]
        question         = s["question"]
        answer           = s["target_json"]

        # Expand each <image> tag with the correct number of IMG_CONTEXT tokens
        # for that specific frame (dynamic tiling means each frame may differ)
        expanded_q = question
        for n_patches in num_patches_list:
            expanded_q = expanded_q.replace(
                IMG_TOKEN,
                IMG_CONTEXT_TOKEN * self.num_image_token * n_patches,
                1   # replace only the first occurrence each iteration
            )

        SYSTEM = ("You are an expert video analyst specialising in financial "
                  "harassment, fraud, abuse, and workplace misconduct. "
                  "Always respond with valid JSON only.")

        full_text = (
            f"<|im_start|>system\n{SYSTEM}<|im_end|>"
            f"<|im_start|>user\n{expanded_q}<|im_end|>"
            f"<|im_start|>assistant\n{answer}<|im_end|>"
        )

        encoding = self.tokenizer(
            full_text,
            return_tensors = "pt",
            max_length     = self.max_length,
            truncation     = True,
            padding        = False,
        )
        input_ids = encoding["input_ids"].squeeze(0)

        # Labels: mask everything up to (and including) assistant header
        labels = input_ids.clone()
        assistant_header = self.tokenizer.encode(
            "<|im_start|>assistant\n", add_special_tokens=False
        )
        header_len = len(assistant_header)
        mask_until = 0
        for pos in range(len(input_ids) - header_len):
            if input_ids[pos : pos + header_len].tolist() == assistant_header:
                mask_until = pos + header_len

        labels[:mask_until] = -100
        pad_id = self.tokenizer.pad_token_id or self.tokenizer.eos_token_id
        labels[labels == pad_id] = -100

        return {
            "input_ids"     : input_ids,
            "attention_mask": torch.ones_like(input_ids),
            "labels"        : labels,
            "pixel_values"  : pixel_values,    # kept as-is for collator
        }


def collate_fn(batch):
    pad_id = tokenizer.pad_token_id or tokenizer.eos_token_id

    input_ids      = pad_sequence([b["input_ids"]      for b in batch], batch_first=True, padding_value=pad_id)
    attention_mask = pad_sequence([b["attention_mask"] for b in batch], batch_first=True, padding_value=0)
    labels         = pad_sequence([b["labels"]         for b in batch], batch_first=True, padding_value=-100)

    # pixel_values: each sample may have different total_tiles
    # Concatenate along dim=0 (tile dim)
    pixel_values = torch.cat([b["pixel_values"] for b in batch], dim=0)

    return {
        "input_ids"     : input_ids,
        "attention_mask": attention_mask,
        "labels"        : labels,
        "pixel_values"  : pixel_values,
    }


train_torch_ds = InternVideo25Dataset(train_samples, tokenizer, model, max_length=MAX_SEQ_LEN)
print(f"✅ PyTorch dataset ready: {len(train_torch_ds)} training samples")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 12 ── Train
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
model.train()
model.language_model.enable_input_require_grads()

training_args = TrainingArguments(
    output_dir                    = OUTPUT_DIR,
    per_device_train_batch_size   = BATCH_SIZE,
    gradient_accumulation_steps   = GRAD_ACCUM,
    num_train_epochs              = EPOCHS,
    learning_rate                 = 2e-4,
    bf16                          = True,
    logging_steps                 = 5,
    save_steps                    = 100,
    save_total_limit              = 2,
    warmup_ratio                  = 0.03,
    lr_scheduler_type             = "cosine",
    optim                         = "paged_adamw_8bit",
    seed                          = RANDOM_SEED,
    remove_unused_columns         = False,
    dataloader_pin_memory         = False,
    gradient_checkpointing        = True,
    gradient_checkpointing_kwargs = {"use_reentrant": False},
    report_to                     = "none",
)

trainer = Trainer(
    model         = model,
    args          = training_args,
    train_dataset = train_torch_ds,
    data_collator = collate_fn,
)

print("\n🚀 Training started...")
trainer.train()
print("✅ Training complete!")

# Save LoRA adapter
model.language_model.save_pretrained("internvideo25_lora_adapter")
tokenizer.save_pretrained("internvideo25_lora_adapter")
os.makedirs(ADAPTER_DRIVE, exist_ok=True)
shutil.copytree("internvideo25_lora_adapter", ADAPTER_DRIVE, dirs_exist_ok=True)
print(f"✅ LoRA adapter saved → {ADAPTER_DRIVE}")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 13 ── Inference
#
#  Official InternVideo2.5 inference:
#    model.chat(tokenizer, pixel_values, question,
#               generation_config, num_patches_list=..., history=None)
#  num_patches_list is MANDATORY — it tells the model how many
#  tiles belong to each frame for correct attention masking.
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
model.eval()

GENERATION_CONFIG = dict(
    do_sample      = False,
    temperature    = 0.0,
    max_new_tokens = 512,
    top_p          = 0.1,
    num_beams      = 1,
)

def predict(video_path: str) -> dict:
    """
    Run InternVideo2.5 inference on a video file.
    Uses decord for frame extraction + dynamic tiling.
    """
    try:
        pixel_values, num_patches_list = load_video_internvideo(
            video_path, num_segments=NUM_SEGMENTS, max_num=MAX_NUM
        )
    except Exception as e:
        print(f"  ⚠️  Decord error: {e}")
        return {}

    # Cast to bfloat16 and move to model device
    pixel_values = pixel_values.to(dtype=torch.bfloat16, device=model.device)

    question = build_question(len(num_patches_list))

    with torch.no_grad():
        response, _ = model.chat(
            tokenizer         = tokenizer,
            pixel_values      = pixel_values,
            question          = question,
            generation_config = GENERATION_CONFIG,
            num_patches_list  = num_patches_list,  # REQUIRED for InternVideo2.5
            history           = None,
            return_history    = True,
        )

    return parse_json_response(response)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 14 ── Run Inference on ALL samples (train + eval)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("\n" + "="*65)
print("  🔍 Running inference on ALL samples (train + eval)")
print("="*65)

all_samples = train_samples + eval_samples
all_results = []

for i, s in enumerate(all_samples):
    video_path = find_video(s["uni_id"])
    pred = predict(video_path)
    tgt  = json.loads(s["target_json"])

    all_results.append({
        "uni_id"          : s["uni_id"],
        "split"           : "train" if i < len(train_samples) else "eval",
        "true_category"   : tgt.get("category",    ""),
        "true_aspect"     : tgt.get("aspect",       ""),
        "true_intent"     : str(tgt.get("intent",   "")),
        "true_explanation": tgt.get("explanation",  ""),
        "true_country"    : tgt.get("country",      ""),
        "true_source"     : tgt.get("source",       ""),
        "true_sector"     : tgt.get("sector",       ""),
        "pred_category"   : str(pred.get("category",    "")).strip(),
        "pred_aspect"     : str(pred.get("aspect",       "")).strip(),
        "pred_intent"     : str(pred.get("intent",       "")).strip(),
        "pred_explanation": str(pred.get("explanation",  "")).strip(),
        "pred_country"    : str(pred.get("country",      "")).strip(),
        "pred_source"     : str(pred.get("source",       "")).strip(),
        "pred_sector"     : str(pred.get("sector",       "")).strip(),
    })

    if (i + 1) % 20 == 0:
        print(f"  Inferred {i+1}/{len(all_samples)}")

print(f"✅ Inference done for {len(all_results)} samples")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 15 ── Accuracy Scoring
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
scorer = rs.RougeScorer(["rougeL"], use_stemmer=True)

def score_split(rows, label="ALL"):
    if not rows:
        return {}

    def acc(t, p):
        return accuracy_score(
            [x.lower().strip() for x in t],
            [x.lower().strip() for x in p]
        )

    rouge_scores = [
        scorer.score(r["true_explanation"], r["pred_explanation"])["rougeL"].fmeasure
        for r in rows
    ]
    avg_rouge = float(np.mean(rouge_scores))

    cat_acc = acc([r["true_category"] for r in rows], [r["pred_category"] for r in rows])
    asp_acc = acc([r["true_aspect"]   for r in rows], [r["pred_aspect"]   for r in rows])
    int_acc = acc([r["true_intent"]   for r in rows], [r["pred_intent"]   for r in rows])
    cty_acc = acc([r["true_country"]  for r in rows], [r["pred_country"]  for r in rows])
    sec_acc = acc([r["true_sector"]   for r in rows], [r["pred_sector"]   for r in rows])
    src_acc = acc([r["true_source"]   for r in rows], [r["pred_source"]   for r in rows])

    overall = (cat_acc*0.25 + asp_acc*0.25 + int_acc*0.15 +
               avg_rouge*0.20 + sec_acc*0.10 + cty_acc*0.03 + src_acc*0.02)

    return {
        "split"             : label,
        "n_samples"         : len(rows),
        "category_acc"      : round(cat_acc   * 100, 2),
        "aspect_acc"        : round(asp_acc   * 100, 2),
        "intent_acc"        : round(int_acc   * 100, 2),
        "country_acc"       : round(cty_acc   * 100, 2),
        "source_acc"        : round(src_acc   * 100, 2),
        "sector_acc"        : round(sec_acc   * 100, 2),
        "explanation_rougeL": round(avg_rouge * 100, 2),
        "OVERALL_SCORE_%"   : round(overall   * 100, 2),
    }


train_rows   = [r for r in all_results if r["split"] == "train"]
eval_rows    = [r for r in all_results if r["split"] == "eval"]
train_scores = score_split(train_rows,  "TRAIN")
eval_scores  = score_split(eval_rows,   "EVAL")
all_scores   = score_split(all_results, "ALL")

def print_scores(s: dict):
    print(f"\n  ── {s['split']} SET  ({s['n_samples']} samples) ──")
    print(f"   Category    Accuracy  : {s['category_acc']:>7.2f} %   (weight 25%)")
    print(f"   Aspect      Accuracy  : {s['aspect_acc']:>7.2f} %   (weight 25%)")
    print(f"   Intent      Accuracy  : {s['intent_acc']:>7.2f} %   (weight 15%)")
    print(f"   Explanation ROUGE-L   : {s['explanation_rougeL']:>7.2f} %   (weight 20%)")
    print(f"   Sector      Accuracy  : {s['sector_acc']:>7.2f} %   (weight 10%)")
    print(f"   Country     Accuracy  : {s['country_acc']:>7.2f} %   (weight  3%)")
    print(f"   Source      Accuracy  : {s['source_acc']:>7.2f} %   (weight  2%)")
    print(f"   {'─'*42}")
    print(f"   ⭐ OVERALL SCORE      : {s['OVERALL_SCORE_%']:>7.2f} %")

print("\n" + "="*65)
print("              📊 ACCURACY REPORT")
print("="*65)
print_scores(train_scores)
print_scores(eval_scores)
print_scores(all_scores)
print("="*65)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 16 ── Save Results to Excel (7 sheets)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
results_df = pd.DataFrame(all_results)
summary_df = pd.DataFrame([train_scores, eval_scores, all_scores])

os.makedirs(os.path.dirname(RESULTS_EXCEL_DRIVE), exist_ok=True)

with pd.ExcelWriter(RESULTS_EXCEL_DRIVE, engine="openpyxl") as writer:
    summary_df.to_excel(writer, sheet_name="Accuracy Summary", index=False)
    results_df.to_excel(writer, sheet_name="Per Sample",       index=False)

    for field in ["category", "aspect", "intent", "sector", "country", "source"]:
        breakdown = results_df[[
            "uni_id", "split",
            f"true_{field}", f"pred_{field}"
        ]].copy()
        breakdown["correct"] = (
            breakdown[f"true_{field}"].str.lower().str.strip() ==
            breakdown[f"pred_{field}"].str.lower().str.strip()
        )
        breakdown.to_excel(writer, sheet_name=f"{field.capitalize()} Detail", index=False)

print(f"\n✅ Results saved → {RESULTS_EXCEL_DRIVE}")
print("\n🎉 ALL DONE!")